In [ ]:
%pip install "transformers>=4.45.0" "torch>=2.1" "torchvision" "Pillow" "tqdm" "datasets>=2.14.6" "gradio>=4.36" "nncf>=2.13.0" --extra-index-url https://download.pytorch.org/whl/cpu
%pip install -U --pre "openvino>=2024.5.0" --extra-index-url https://storage.openvinotoolkit.org/simple/wheels/nightly

In [ ]:
from pathlib import Path
from ov_mllama_helper import convert_mllama

model_id = "Llama-3.2-11B-Vision-Instruct"
model_dir = Path(model_id.split("/")[-1]) / "OV"

# uncomment the line to see model conversion code 
# convert_mllama??

In [ ]:
convert_mllama(model_id, model_dir)

In [ ]:
from notebook_utils import device_widget

device = device_widget()

In [ ]:
from transformers import AutoProcessor
from ov_mllama_compression import compress

processor = AutoProcessor.from_pretrained(model_dir)

# uncomment the line to see compression code
# compress??

In [ ]:
compress(model_dir, processor)